### Loading Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp

### Loading Dataset 

In [2]:
# Load the data
file_path = 'final_data.csv'
stock_data = pd.read_csv(file_path)

# Check for and drop the 'Unnamed: 0' column if it exists
if 'Unnamed: 0' in stock_data.columns:
    stock_data.drop(columns=['Unnamed: 0'], inplace=True)
    
stock_data.head()

,Adj Close,Volume,Date,Symbol,Year,Outstanding_Shares
0,429.1,1411814,01/01/2014,MCX,2014,50.998
1,424.3,2919045,02/01/2014,MCX,2014,50.998
2,499.6,8191055,03/01/2014,MCX,2014,50.998
3,497.8,8395828,06/01/2014,MCX,2014,50.998
4,517.0,6823517,07/01/2014,MCX,2014,50.998


### Data Cleaning

In [3]:
# Convert 'Outstanding_Shares' to numeric (if not already)
stock_data['Outstanding_Shares'] = pd.to_numeric(stock_data['Outstanding_Shares'], errors='coerce')

# Multiply 'Outstanding_Shares' by 10,000,000 (since it's in crores) before calculating turnover
stock_data['Outstanding_Shares'] *= 10000000

# Calculate daily turnover
stock_data['Turnover'] = stock_data['Volume'] / stock_data['Outstanding_Shares']

# Convert 'Date' to datetime format
stock_data['Date'] = pd.to_datetime(stock_data['Date'], format="%d/%m/%Y")

# Set 'Date' as the index
stock_data.set_index('Date', inplace=True)

# Display the first few rows with the new Turnover column
stock_data.head()

,Adj Close,Volume,Symbol,Year,Outstanding_Shares,Turnover
Date,,,,,,
2014-01-01,429.1,1411814,MCX,2014,509980000.0,0.002768
2014-01-02,424.3,2919045,MCX,2014,509980000.0,0.005724
2014-01-03,499.6,8191055,MCX,2014,509980000.0,0.016062
2014-01-06,497.8,8395828,MCX,2014,509980000.0,0.016463
2014-01-07,517.0,6823517,MCX,2014,509980000.0,0.013380


#### Calculating J Month Past Returns 

In [4]:
# Resample to monthly frequency to get the first price of each month
monthly_data = stock_data.groupby('Symbol').resample('BMS').first()

# Reset the index to make 'Symbol' and 'Date' columns again
monthly_data = monthly_data.reset_index(level=0, drop=True)

# Calculate monthly returns in percentage
monthly_data['Monthly_Return'] = monthly_data.groupby('Symbol')['Adj Close'].pct_change() * 100

# Calculate cumulative returns for 3, 6, 9, and 12 months in percentage
monthly_data['3M_Return'] = monthly_data.groupby('Symbol')['Adj Close'].pct_change(3) * 100
monthly_data['6M_Return'] = monthly_data.groupby('Symbol')['Adj Close'].pct_change(6) * 100
monthly_data['9M_Return'] = monthly_data.groupby('Symbol')['Adj Close'].pct_change(9) * 100
monthly_data['12M_Return'] = monthly_data.groupby('Symbol')['Adj Close'].pct_change(12) * 100

monthly_data.reset_index(inplace=True)
monthly_data.head()

,Date,Adj Close,Volume,Symbol,Year,Outstanding_Shares,Turnover,Monthly_Return,3M_Return,6M_Return,9M_Return,12M_Return
0,2014-01-01,3445.1,149,3MINDIA,2014,112650000.0,1.322681e-06,NaN,NaN,NaN,NaN,NaN
1,2014-02-03,3443.1,76,3MINDIA,2014,112650000.0,6.746560e-07,-0.058053,NaN,NaN,NaN,NaN
2,2014-03-03,3461.4,419,3MINDIA,2014,112650000.0,3.719485e-06,0.531498,NaN,NaN,NaN,NaN
3,2014-04-01,3404.7,253,3MINDIA,2014,112650000.0,2.245894e-06,-1.638066,-1.172680,NaN,NaN,NaN
4,2014-05-01,3550.6,239,3MINDIA,2014,112650000.0,2.121616e-06,4.285253,3.122186,NaN,NaN,NaN


#### Ranking Stocks based on Past J Month Returns 

In [5]:
# Rank stocks based on cumulative returns
for period in ['3M', '6M', '9M', '12M']:
    rank_col = f'{period}_Rank'
    monthly_data[rank_col] = monthly_data.groupby('Date')[f'{period}_Return'].rank(method='first', ascending=False)

monthly_data.head()

,Date,Adj Close,Volume,Symbol,Year,Outstanding_Shares,Turnover,Monthly_Return,3M_Return,6M_Return,9M_Return,12M_Return,3M_Rank,6M_Rank,9M_Rank,12M_Rank
0,2014-01-01,3445.1,149,3MINDIA,2014,112650000.0,1.322681e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-02-03,3443.1,76,3MINDIA,2014,112650000.0,6.746560e-07,-0.058053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-03-03,3461.4,419,3MINDIA,2014,112650000.0,3.719485e-06,0.531498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-04-01,3404.7,253,3MINDIA,2014,112650000.0,2.245894e-06,-1.638066,-1.172680,NaN,NaN,NaN,253.0,NaN,NaN,NaN
4,2014-05-01,3550.6,239,3MINDIA,2014,112650000.0,2.121616e-06,4.285253,3.122186,NaN,NaN,NaN,293.0,NaN,NaN,NaN


#### Assign Ranks to R1-R10 Quintile

In [6]:
# Define quintile function with check for sufficient unique values
def assign_quintile(x, rank_col, quintile_col):
    if len(x[rank_col].unique()) < 10:
        x[quintile_col] = np.nan
    else:
        x[quintile_col] = pd.qcut(x[rank_col], 10, labels=False) + 1
    return x

# Apply quintile ranking
for period in ['3M', '6M', '9M', '12M']:
    rank_col = f'{period}_Rank'
    quintile_col = f'{period}_Quintile'
    monthly_data = monthly_data.groupby('Date').apply(assign_quintile, rank_col, quintile_col).reset_index(drop=True)

monthly_data[monthly_data['Symbol'] == 'ABB'].head()#### Calculating J Month Past Returns 

,Date,Adj Close,Volume,Symbol,Year,Outstanding_Shares,Turnover,Monthly_Return,3M_Return,6M_Return,9M_Return,12M_Return,3M_Rank,6M_Rank,9M_Rank,12M_Rank,3M_Quintile,6M_Quintile,9M_Quintile,12M_Quintile
2,2014-01-01,591.9,75730,ABB,2014,2.119080e+09,0.000036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339,2014-02-03,496.2,138871,ABB,2014,2.119080e+09,0.000066,-16.168272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
676,2014-03-03,620.9,392148,ABB,2014,2.119080e+09,0.000185,25.130996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,2014-04-01,732.8,180312,ABB,2014,2.119080e+09,0.000085,18.022226,23.804697,NaN,NaN,NaN,61.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
1350,2014-05-01,724.1,121174,ABB,2014,2.119080e+09,0.000057,-1.187227,45.929061,NaN,NaN,NaN,43.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN


#### Calculate Average Turnover over the J formation periods

In [7]:
# Calculate average turnover over the formation periods
for period in ['3M', '6M', '9M', '12M']:
    avg_turnover_col = f'{period}_Avg_Turnover'
    window_size = int(period[:-1])
    monthly_data[avg_turnover_col] = monthly_data.groupby('Symbol')['Turnover'].rolling(window=window_size,min_periods=1).mean().reset_index(level=0, drop=True)

monthly_data.head()

,Date,Adj Close,Volume,Symbol,Year,Outstanding_Shares,Turnover,Monthly_Return,3M_Return,6M_Return,...,9M_Rank,12M_Rank,3M_Quintile,6M_Quintile,9M_Quintile,12M_Quintile,3M_Avg_Turnover,6M_Avg_Turnover,9M_Avg_Turnover,12M_Avg_Turnover
0,2014-01-01,3445.1,149,3MINDIA,2014,1.126500e+08,0.000001,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000001,0.000001,0.000001,0.000001
1,2014-01-01,22.2,94557,AARTIIND,2014,3.332820e+09,0.000028,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000028,0.000028,0.000028,0.000028
2,2014-01-01,591.9,75730,ABB,2014,2.119080e+09,0.000036,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000036,0.000036,0.000036,0.000036
3,2014-01-01,1549.1,1444,ABBOTINDIA,2014,2.124900e+08,0.000007,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000007,0.000007,0.000007,0.000007
4,2014-01-01,87.5,9623,ABFRL,2014,9.413900e+08,0.000010,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000010,0.000010,0.000010,0.000010


#### Ranking Stocks Based on Average Turnover

In [8]:
# Rank stocks based on average turnover
for period in ['3M', '6M', '9M', '12M']:
    avg_turnover_col = f'{period}_Avg_Turnover'
    rank_col = f'{period}_Turnover_Rank'
    monthly_data[rank_col] = monthly_data.groupby('Date')[avg_turnover_col].rank(method='first')
    
monthly_data.head()

,Date,Adj Close,Volume,Symbol,Year,Outstanding_Shares,Turnover,Monthly_Return,3M_Return,6M_Return,...,9M_Quintile,12M_Quintile,3M_Avg_Turnover,6M_Avg_Turnover,9M_Avg_Turnover,12M_Avg_Turnover,3M_Turnover_Rank,6M_Turnover_Rank,9M_Turnover_Rank,12M_Turnover_Rank
0,2014-01-01,3445.1,149,3MINDIA,2014,1.126500e+08,0.000001,NaN,NaN,NaN,...,NaN,NaN,0.000001,0.000001,0.000001,0.000001,11.0,11.0,11.0,11.0
1,2014-01-01,22.2,94557,AARTIIND,2014,3.332820e+09,0.000028,NaN,NaN,NaN,...,NaN,NaN,0.000028,0.000028,0.000028,0.000028,125.0,125.0,125.0,125.0
2,2014-01-01,591.9,75730,ABB,2014,2.119080e+09,0.000036,NaN,NaN,NaN,...,NaN,NaN,0.000036,0.000036,0.000036,0.000036,145.0,145.0,145.0,145.0
3,2014-01-01,1549.1,1444,ABBOTINDIA,2014,2.124900e+08,0.000007,NaN,NaN,NaN,...,NaN,NaN,0.000007,0.000007,0.000007,0.000007,41.0,41.0,41.0,41.0
4,2014-01-01,87.5,9623,ABFRL,2014,9.413900e+08,0.000010,NaN,NaN,NaN,...,NaN,NaN,0.000010,0.000010,0.000010,0.000010,58.0,58.0,58.0,58.0


#### Assign Average Turnover Ranks to Terciles V1-V3

In [9]:
# Define tercile function with check for sufficient unique values
def assign_tercile(x, rank_col, tercile_col):
    if len(x[rank_col].unique()) < 3:
        x[tercile_col] = np.nan
    else:
        x[tercile_col] = pd.qcut(x[rank_col], 3, labels=False) + 1
    return x

# Apply tercile ranking
for period in ['3M', '6M', '9M', '12M']:
    rank_col = f'{period}_Turnover_Rank'
    tercile_col = f'{period}_Turnover_Tercile'
    monthly_data = monthly_data.groupby('Date').apply(assign_tercile, rank_col, tercile_col).reset_index(drop=True)
    
monthly_data.head()

,Date,Adj Close,Volume,Symbol,Year,Outstanding_Shares,Turnover,Monthly_Return,3M_Return,6M_Return,...,9M_Avg_Turnover,12M_Avg_Turnover,3M_Turnover_Rank,6M_Turnover_Rank,9M_Turnover_Rank,12M_Turnover_Rank,3M_Turnover_Tercile,6M_Turnover_Tercile,9M_Turnover_Tercile,12M_Turnover_Tercile
0,2014-01-01,3445.1,149,3MINDIA,2014,1.126500e+08,0.000001,NaN,NaN,NaN,...,0.000001,0.000001,11.0,11.0,11.0,11.0,1,1,1,1
1,2014-01-01,22.2,94557,AARTIIND,2014,3.332820e+09,0.000028,NaN,NaN,NaN,...,0.000028,0.000028,125.0,125.0,125.0,125.0,2,2,2,2
2,2014-01-01,591.9,75730,ABB,2014,2.119080e+09,0.000036,NaN,NaN,NaN,...,0.000036,0.000036,145.0,145.0,145.0,145.0,2,2,2,2
3,2014-01-01,1549.1,1444,ABBOTINDIA,2014,2.124900e+08,0.000007,NaN,NaN,NaN,...,0.000007,0.000007,41.0,41.0,41.0,41.0,1,1,1,1
4,2014-01-01,87.5,9623,ABFRL,2014,9.413900e+08,0.000010,NaN,NaN,NaN,...,0.000010,0.000010,58.0,58.0,58.0,58.0,1,1,1,1


#### Calculate Current Month (1 Month Future Return) 

In [10]:
monthly_data['Current_Month_Return'] = monthly_data.groupby('Symbol')['Monthly_Return'].shift(-1)

#### PRICE MOMENTUM RESULTS

In [11]:
# Function to calculate the equal-weighted average return for a portfolio
def calculate_weighted_avg_returns(df, J, K, rank):
    df['YearMonth'] = df['Date'].dt.to_period('M')
    unique_year_months = df['YearMonth'].unique()
    portfolio_returns = []

    for current_year_month in unique_year_months:
        returns = 0
        valid_count = 0

        for offset in range(K):
            # Ensure we use exact YearMonth dates from the dataset
            index = np.where(unique_year_months == current_year_month)[0][0] - offset
            if index < 0:
                continue
            look_back_year_month = unique_year_months[index]
            portfolio_stocks = df[(df['YearMonth'] == look_back_year_month) & (df[f'{J}M_Quintile'] == rank)]['Symbol']

            if not portfolio_stocks.empty:
                current_returns = df[(df['YearMonth'] == current_year_month) & (df['Symbol'].isin(portfolio_stocks))]['Current_Month_Return'].mean()
                if not np.isnan(current_returns):
                    returns += current_returns
                    valid_count += 1         

        if valid_count > 0:
            average_return = returns / valid_count
            portfolio_returns.append({'YearMonth': current_year_month, 'J': J, 'K': K, 'Rank': 'R'+str(rank), 'Portfolio_Return': average_return})

    return pd.DataFrame(portfolio_returns)

# Calculate the weighted average returns for all combinations of J, K, and ranks
results = []
for J in [3, 6, 9, 12]:
    for K in [3, 6, 9, 12]:
        for rank in [1, 10]:
            result = calculate_weighted_avg_returns(monthly_data, J, K, rank)
            results.append(result)

# Combine all results into a single DataFrame
portfolio_returns_df = pd.concat(results, ignore_index=True)

portfolio_returns_df

,YearMonth,J,K,Rank,Portfolio_Return
0,2014-04,3,3,R1,6.376619
1,2014-05,3,3,R1,25.586237
2,2014-06,3,3,R1,15.586128
3,2014-07,3,3,R1,-4.336001
4,2014-08,3,3,R1,7.584021
...,...,...,...,...,...
3563,2023-07,12,12,R10,8.567848
3564,2023-08,12,12,R10,8.586148
3565,2023-09,12,12,R10,2.213409
3566,2023-10,12,12,R10,-4.655477


##### Consolidate Results For Each J and K combination ( 4 J * 4 K * 3 (R1, R10, R1-R10) = 48 combinations)

In [12]:
# Function to calculate mean and t-test for each group
def calculate_mean_and_ttest(group):
    mean_return = group['Portfolio_Return'].mean()
    t_stat, _ = ttest_1samp(group['Portfolio_Return'], 0)
    return pd.Series({'Mean_Return': mean_return, 'T_Value': t_stat})

# Function to calculate Rank 1 - Rank 10 returns and t-test
def calculate_diff_and_ttest(df):
    rank1 = df[df['Rank'] == 1]
    rank10 = df[df['Rank'] == 10]
    
    if not rank1.empty and not rank10.empty:
        diff_returns = rank1['Portfolio_Return'].values - rank10['Portfolio_Return'].values
        mean_diff = diff_returns.mean()
        t_stat, _ = ttest_1samp(diff_returns, 0)
        return pd.Series({'Mean_Return': mean_diff, 'T_Value': t_stat, 'Rank': 'R1 - R10'})
    else:
        return pd.Series({'Mean_Return': None, 'T_Value': None, 'Rank': 'R1 - R10'})

# Group by J, K, and Rank and apply the function
mean_portfolio_returns_df = portfolio_returns_df.groupby(['J', 'K', 'Rank']).apply(calculate_mean_and_ttest).reset_index()

# Calculate Rank 1 - Rank 10 difference and t-test
diff_returns_df = portfolio_returns_df.groupby(['J', 'K']).apply(calculate_diff_and_ttest).reset_index()

# Combine the results
combined_df = pd.concat([mean_portfolio_returns_df, diff_returns_df], ignore_index=True)

combined_df

,J,K,Rank,Mean_Return,T_Value
0,3,3,R1,3.636688,5.201133
1,3,3,R10,2.191973,3.275026
2,3,6,R1,3.520713,5.211864
3,3,6,R10,2.178659,3.274775
4,3,9,R1,3.405942,5.244520
5,3,9,R10,2.241708,3.298017
6,3,12,R1,3.300450,5.086674
7,3,12,R10,2.313987,3.376173
8,6,3,R1,3.397601,5.071108
9,6,3,R10,1.738763,2.464670


#### PRICE MOMENTUM AND TURNOVER RESULTS

In [13]:
# Function to calculate the equal-weighted average return for a portfolio including volume rankings
def calculate_weighted_avg_returns(df, J, K, return_rank, volume_rank):
    df['YearMonth'] = df['Date'].dt.to_period('M')
    unique_year_months = df['YearMonth'].unique()
    portfolio_returns = []

    for current_year_month in unique_year_months:
        returns = 0
        valid_count = 0

        for offset in range(K):
            # Ensure we use exact YearMonth dates from the dataset
            index = np.where(unique_year_months == current_year_month)[0][0] - offset
            if index < 0:
                continue
            look_back_year_month = unique_year_months[index]
            portfolio_stocks = df[(df['YearMonth'] == look_back_year_month) & (df[f'{J}M_Quintile'] == return_rank) & (df[f'{J}M_Turnover_Tercile'] == volume_rank)]['Symbol']

            if not portfolio_stocks.empty:
                current_returns = df[(df['YearMonth'] == current_year_month) & (df['Symbol'].isin(portfolio_stocks))]['Current_Month_Return'].mean()
                if not np.isnan(current_returns):
                    returns += current_returns
                    valid_count += 1

        if valid_count > 0:
            average_return = returns / valid_count
            portfolio_returns.append({'YearMonth': current_year_month, 'J': J, 'K': K, 'Return_Rank': 'R'+str(return_rank), 'Volume_Rank': 'V'+str(volume_rank), 'Portfolio_Return': average_return})

    return pd.DataFrame(portfolio_returns)

# Calculate the weighted average returns for all combinations of J, K, return ranks, and volume ranks
results = []
for J in [3, 6, 9, 12]:
    for K in [3, 6, 9, 12]:
        for return_rank in [1, 10]:
            for volume_rank in [1, 2, 3]:  # 1 for lowest volume tercile, 3 for highest volume tercile
                result = calculate_weighted_avg_returns(monthly_data, J, K, return_rank, volume_rank)
                results.append(result)

# Combine all results into a single DataFrame
portfolio_returns_df = pd.concat(results, ignore_index=True)

# Display the result
portfolio_returns_df

,YearMonth,J,K,Return_Rank,Volume_Rank,Portfolio_Return
0,2014-04,3,3,R1,V1,5.691872
1,2014-05,3,3,R1,V1,29.131867
2,2014-06,3,3,R1,V1,9.643532
3,2014-07,3,3,R1,V1,-6.280164
4,2014-08,3,3,R1,V1,1.056106
...,...,...,...,...,...,...
10699,2023-07,12,12,R10,V3,9.861298
10700,2023-08,12,12,R10,V3,8.131906
10701,2023-09,12,12,R10,V3,1.596635
10702,2023-10,12,12,R10,V3,-4.027368


#### Add Records for V1-V3 for each J, K, R1, R10

In [14]:
# Add records for V1 - V3 for each J, K, R1, R10
diff_results = []

for (J, K, return_rank), group in portfolio_returns_df.groupby(['J', 'K', 'Return_Rank']):
    v1 = group[group['Volume_Rank'] == 'V1']
    v3 = group[group['Volume_Rank'] == 'V3']
    
    if not v1.empty and not v3.empty:
        for date in v1['YearMonth'].unique():
            v1_return = v1[v1['YearMonth'] == date]['Portfolio_Return'].values
            v3_return = v3[v3['YearMonth'] == date]['Portfolio_Return'].values
            if len(v1_return) > 0 and len(v3_return) > 0:
                diff_returns_v3_v1 = v3_return[0] - v1_return[0]
                diff_results.append({'YearMonth': date, 'J': J, 'K': K, 'Return_Rank': return_rank, 'Volume_Rank': 'V3-V1', 'Portfolio_Return': diff_returns_v3_v1})
                
# Convert the diff_results list to a DataFrame and concatenate with portfolio_returns_df
diff_results_df = pd.DataFrame(diff_results)
portfolio_returns_df = pd.concat([portfolio_returns_df, diff_results_df], ignore_index=True)
portfolio_returns_df

,YearMonth,J,K,Return_Rank,Volume_Rank,Portfolio_Return
0,2014-04,3,3,R1,V1,5.691872
1,2014-05,3,3,R1,V1,29.131867
2,2014-06,3,3,R1,V1,9.643532
3,2014-07,3,3,R1,V1,-6.280164
4,2014-08,3,3,R1,V1,1.056106
...,...,...,...,...,...,...
14267,2023-07,12,12,R10,V3-V1,1.118496
14268,2023-08,12,12,R10,V3-V1,-2.930740
14269,2023-09,12,12,R10,V3-V1,-1.517523
14270,2023-10,12,12,R10,V3-V1,-0.988798


#### Add records for R1 - R10 for each J, K, V1, V2, V3, V3-V1

In [15]:
# Add records for R1 - R10 for each J, K, V1, V2, V3
for (J, K, volume_rank), group in portfolio_returns_df.groupby(['J', 'K', 'Volume_Rank']):
    rank1 = group[group['Return_Rank'] == 'R1']
    rank10 = group[group['Return_Rank'] == 'R10']
    
    if not rank1.empty and not rank10.empty:
        for date in rank1['YearMonth'].unique():
            rank1_return = rank1[rank1['YearMonth'] == date]['Portfolio_Return'].values
            rank10_return = rank10[rank10['YearMonth'] == date]['Portfolio_Return'].values
            if len(rank1_return) > 0 and len(rank10_return) > 0:
                diff_returns_r1_r10 = rank1_return[0] - rank10_return[0]
                diff_results.append({'YearMonth': date, 'J': J, 'K': K, 'Return_Rank': 'R1-R10', 'Volume_Rank': volume_rank, 'Portfolio_Return': diff_returns_r1_r10})

# Convert the diff_results list to a DataFrame and concatenate with portfolio_returns_df
diff_results_df = pd.DataFrame(diff_results)
portfolio_returns_df = pd.concat([portfolio_returns_df, diff_results_df], ignore_index=True)
portfolio_returns_df

,YearMonth,J,K,Return_Rank,Volume_Rank,Portfolio_Return
0,2014-04,3,3,R1,V1,5.691872
1,2014-05,3,3,R1,V1,29.131867
2,2014-06,3,3,R1,V1,9.643532
3,2014-07,3,3,R1,V1,-6.280164
4,2014-08,3,3,R1,V1,1.056106
...,...,...,...,...,...,...
24971,2023-07,12,12,R1-R10,V3-V1,4.063710
24972,2023-08,12,12,R1-R10,V3-V1,-2.679523
24973,2023-09,12,12,R1-R10,V3-V1,-2.259762
24974,2023-10,12,12,R1-R10,V3-V1,-1.152854


##### Consolidate Results For Each J and K combination ( 4 J * 4 K * 3 (R1, R10, R1-R10) * 4* (V1, V2, V3, V3-V1) = 192 combinations)

In [16]:
# Apply the function to calculate mean return and t-statistic for all combinations
final_results_df = portfolio_returns_df.groupby(['J', 'K', 'Return_Rank', 'Volume_Rank']).apply(calculate_mean_and_ttest).reset_index()

final_results_df

,J,K,Return_Rank,Volume_Rank,Mean_Return,T_Value
0,3,3,R1,V1,4.143818,5.150902
1,3,3,R1,V2,3.450880,4.758338
2,3,3,R1,V3,3.635378,4.720153
3,3,3,R1,V3-V1,-0.508440,-1.054107
4,3,3,R1-R10,V1,1.500572,2.251873
...,...,...,...,...,...,...
187,12,12,R1-R10,V3-V1,-0.116421,-0.164219
188,12,12,R10,V1,2.465994,3.010410
189,12,12,R10,V2,2.255934,2.576689
190,12,12,R10,V3,2.126726,2.450052
